In [15]:
import numpy as np
import openai
import pandas as pd
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys
import re
import time
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# Get the current directory of the notebook
current_dir = os.path.dirname(os.path.abspath("../../data/amazon-beauty/rating_prediction.ipynb"))
print(f"current directory: {current_dir}")

current directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty


# RMSE & MAE evaluation metrics

In [16]:
# Test
actual_ratings = [4, 4]  # Ground truth ratings
predicted_ratings = [3, 5]  # Predicted ratings
rmse, mae = calculate_rmse_and_mae(actual_ratings, predicted_ratings)
print("RMSE: ", rmse)
print("MAE: ", mae)


RMSE:  1.0
MAE:  1.0


# Data Overview

In [17]:
# Construct the path to data file
data_path = os.path.join(current_dir, 'merged_data.csv')
print(f'data path: {data_path}')

data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/merged_data.csv


In [18]:
# Read the data
data = pd.read_csv(data_path)
# get necessary columns
# data = data[['title', 'rating', 'reviewText', 'reviewerID']]
# get sample data of NUM_SAMPLES rows
data.info()
data.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rating           34 non-null     float64
 1   verified         34 non-null     bool   
 2   reviewTime       34 non-null     object 
 3   reviewerID       34 non-null     object 
 4   asin             34 non-null     object 
 5   reviewerName     34 non-null     object 
 6   reviewText       34 non-null     object 
 7   summary          34 non-null     object 
 8   unixReviewTime   34 non-null     int64  
 9   vote             3 non-null      float64
 10  style            17 non-null     object 
 11  image            0 non-null      float64
 12  category         34 non-null     object 
 13  tech1            0 non-null      float64
 14  description      34 non-null     object 
 15  fit              0 non-null      float64
 16  title            34 non-null     object 
 17  also_buy         3

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,feature,rank,also_view,details,main_cat,similar_item,date,price,imageURL,imageURLHighRes
0,5.0,True,"09 17, 2015",ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,1442448000,NaN,...,[],"120,123 in Beauty & Personal Care (","['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,$5.17,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1,5.0,True,"09 17, 2015",ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,1442448000,NaN,...,[],"120,123 in Beauty & Personal Care (","['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,$5.17,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
2,2.0,True,"03 27, 2018",A2TU781PWGS09X,B00006L9LC,Amazon Customer,Doesnt smell,Two Stars,1522108800,NaN,...,[],"1,693,702 in Beauty & Personal Care (",[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,$23.00,[],[]


# Zero-shot (OpenAI API)

+ We used the ``.drop_duplicates()`` method to get unique pairs of "title" and "reviewText". The predictions are then based on both the title and the corresponding review text for each unique pair.

In [ ]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_training=['title', 'reviewText'],
                                       columns_for_prediction=['title'],
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path='../../data/amazon-beauty/reviewText_small_predictions_zero_shot.csv')
# read csv file
merged_data_with_predictions = pd.read_csv('../../data/amazon-beauty/reviewText_small_predictions_zero_shot.csv')
merged_data_with_predictions.head(3)


In [23]:
# Columns related to the item side can be used for the prediction
ITEM_SIDE = [
    "asin",
    "title",
    "feature",
    "description",
    "price",
    "imageURL",
    "imageURLHighRes",
    "brand",
    "category",
    "tech1",
    "tech2",
    "also_buy",
    "also_view",
    "details",
    "main_cat",
    "similar_item",
    "date",
    "rank"
]

# Columns related to the interaction side can't be used for the prediction
INTERACTION_SIDE = [
    "reviewText",
    "rating",
    "summary",
    "unixReviewTime",
    "reviewTime",
    "vote",
    "style",
    "image"
]

# Columns related to the user side can't be used for the prediction
USER_SIDE = [
    "reviewerID",
    "reviewerName",
    "verified"
]


In [24]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_training=ITEM_SIDE + INTERACTION_SIDE + USER_SIDE,  # Use all for training
                                       columns_for_prediction=ITEM_SIDE,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path='../../data/amazon-beauty/reviewText_small_predictions_zero_shot.csv')
# read csv file
merged_data_with_predictions = pd.read_csv('../../data/amazon-beauty/reviewText_small_predictions_zero_shot.csv')
merged_data_with_predictions.head(3)


No matching data found for ['B000052YAN'
 'Reach Dentotape Waxed Dental Floss with Extra Wide Cleaning Surface for Large Spaces between Teeth, Unflavored, 100 Yards'
 '[]'
 '["Extra wide cleaning surface. ADA Accepted - American Dental Association. The floss dentists use most. Dentotape ribbon floss gives you extra cleaning surface for effective plaque removal. Flossing has been clinically proven to clean away the plaque from between teeth that brushing and rinsing can\'t remove. Made in USA."]'
 '$5.17'
 "['https://images-na.ssl-images-amazon.com/images/I/41EXSAekYdL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/31W%2ByvNqqEL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41lSv8Hs6aL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41CEqMk8%2BEL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41FMSXjSglL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/31UWbJbnZCL._SS40_.jpg']"
 "['https://images-na.ssl-images

ValueError: All arrays must be of the same length

In [21]:
def predict_ratings_zero_shot_and_save(data,
                                       columns_for_training=ITEM_SIDE + INTERACTION_SIDE + USER_SIDE,  # Use all for training
                                       columns_for_prediction=ITEM_SIDE,  # Use only item attributes for prediction
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path='../../data/amazon-beauty/reviewText_large_predictions_zero_shot.csv'):
    """
    This function predicts product ratings using a zero-shot approach based on item attributes
    and saves the predictions to a specified CSV file.

    Parameters:
    - data: DataFrame containing the product reviews data.
    - columns_for_training: List of columns used to uniquely identify each product for training. 
                            Includes ITEM_SIDE, INTERACTION_SIDE, and USER_SIDE.
    - columns_for_prediction: List of columns related to item attributes used for predicting ratings.
    - pause_every_n_users: The function will pause for a specified duration after processing a given number of users. 
                            This can be useful to avoid hitting API rate limits.
    - sleep_time: Duration (in seconds) for which the function should pause.
    - save_path: Path where the predictions will be saved as a CSV file.
    """

    # Nested function to predict ratings by combining provided arguments into a single string
    def predict_rating_zero_shot_with_review(*args):
        """Combine the arguments into a text and use them to predict the rating."""
        combined_text = ". ".join([f"{columns_for_prediction[i]}: {args[i]}" for i in range(len(args)) if args[i] is not None and args[i] != ''])
        return predict_rating_zero_shot_ChatCompletion(combined_text)

    # List to store predicted ratings
    predicted_ratings = []

    # Get unique pairs of products based on columns_for_training
    unique_pairs_for_training = data[columns_for_training].drop_duplicates().values

    # Loop through each unique product pair
    for idx, unique_pair in enumerate(unique_pairs_for_training):

        # Extract the row in the original data that matches the current unique pair
        mask = (data[columns_for_training].values == unique_pair).all(axis=1)
        matching_rows = data[mask]

        # Proceed only if there are matching rows
        if not matching_rows.empty:
            matching_row = matching_rows.iloc[0]

            # Get the data used for prediction
            prediction_data = matching_row[columns_for_prediction].values

            # Predict the rating
            predicted_rating = predict_rating_zero_shot_with_review(*prediction_data)
            print(f"Predicted rating for {unique_pair}: {predicted_rating}")

            # Append the predicted rating to the list
            predicted_ratings.append(predicted_rating)
        else:
            print(f"No matching data found for {unique_pair}. Skipping...")
            continue

        # Pause for sleep_time seconds after processing pause_every_n_users products
        if (idx + 1) % pause_every_n_users == 0:
            print(f"Pausing for {sleep_time} seconds...")
            time.sleep(sleep_time)

    # Create a DataFrame to store the unique product pairs and their predicted ratings
    columns_dict = {columns_for_training[i]: unique_pairs_for_training[:, i] for i in range(len(columns_for_training))}
    columns_dict['zero_shot_predicted_rating'] = predicted_ratings
    predicted_ratings_df = pd.DataFrame(columns_dict)

    # Merge the original data with the predicted ratings
    merged_data_with_predictions = pd.merge(data, predicted_ratings_df, on=columns_for_training)

    # Save the merged data with predictions to the specified path
    merged_data_with_predictions.to_csv(save_path, index=False)


In [22]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_training=ITEM_SIDE + INTERACTION_SIDE + USER_SIDE,  # Use all for training
                                       columns_for_prediction=ITEM_SIDE,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path='../../data/amazon-beauty/all_small_predictions_zero_shot.csv')
# read csv file
merged_data_with_predictions = pd.read_csv('../../data/amazon-beauty/all_small_predictions_zero_shot.csv')
merged_data_with_predictions.head(3)


KeyError: "['related', 'salesRank', 'categories', 'similar'] not in index"

In [ ]:
# evaluate the rating prediction model

product_titles = merged_data_with_predictions['title']
actual_ratings = merged_data_with_predictions['rating']
predicted_ratings = merged_data_with_predictions['predicted_rating']

# Remove None predictions if any
actual_ratings_filtered, predicted_ratings_filtered = zip(*[(actual, predicted) for actual, predicted in zip(actual_ratings, predicted_ratings) if predicted is not None])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(actual_ratings_filtered, predicted_ratings_filtered))
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate MAE
mae = mean_absolute_error(actual_ratings_filtered, predicted_ratings_filtered)
print(f'Mean Absolute Error (MAE): {mae}')


# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

In [ ]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=['title', 'reviewText'],
                                      columns_for_prediction=['title'],
                                      obs_per_user=None,
                                      pause_every_n_users=PAUSE_EVERY_N_USERS,
                                      sleep_time=SLEEP_TIME,
                                      save_path='../../data/amazon-beauty/reviewText_small_predictions_few_shot.csv.csv')
                                      
# load data from ../../data/amazon-beauty/small_predictions_few_shot.csv file
small_predictions_few_shot = pd.read_csv('../../data/amazon-beauty/reviewText_small_predictions_few_shot.csv')
small_predictions_few_shot.head(NUM_EXAMPLES)


In [ ]:
# convert few_shot_predicted_rating column to list
predicted_ratings = small_predictions_few_shot['few_shot_predicted_rating'].tolist()
# convert actual_rating column to list
actual_ratings = small_predictions_few_shot['actual_rating'].tolist()
filtered_list = [(actual, predicted) for actual, predicted in zip(actual_ratings, predicted_ratings) if predicted is not None]

if not filtered_list:
    print("No valid predictions available for evaluation.")
else:
    actual_ratings_filtered, predicted_ratings_filtered = zip(*filtered_list)
    # Evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(actual_ratings_filtered, predicted_ratings_filtered))
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    mae = mean_absolute_error(actual_ratings_filtered, predicted_ratings_filtered)
    print(f'Mean Absolute Error (MAE): {mae}')


# 1 observation per reviewer - Few-shot OpenAI

In [ ]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=['title', 'reviewText'],
                                      columns_for_prediction=['title'],
                                      obs_per_user=1,
                                      pause_every_n_users=PAUSE_EVERY_N_USERS,
                                      sleep_time=SLEEP_TIME,
                                      save_path='../../data/amazon-beauty/reviewText_small_1_test_predictions_few_shot.csv')

small_predictions_few_shot = pd.read_csv('../../data/amazon-beauty/reviewText_small_1_test_predictions_few_shot.csv')
small_predictions_few_shot.head(NUM_EXAMPLES)

In [ ]:

# convert few_shot_predicted_rating column to list
predicted_ratings = small_predictions_few_shot['few_shot_predicted_rating'].tolist()
# convert actual_rating column to list
actual_ratings = small_predictions_few_shot['actual_rating'].tolist()
filtered_list = [(actual, predicted) for actual, predicted in zip(actual_ratings, predicted_ratings) if predicted is not None]

if not filtered_list:
    print("No valid predictions available for evaluation.")
else:
    actual_ratings_filtered, predicted_ratings_filtered = zip(*filtered_list)
    # Evaluate the model's performance
    rmse = np.sqrt(mean_squared_error(actual_ratings_filtered, predicted_ratings_filtered))
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    mae = mean_absolute_error(actual_ratings_filtered, predicted_ratings_filtered)
    print(f'Mean Absolute Error (MAE): {mae}')


# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a small number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication